# El problema de los misioneros y los caníbales en AIMA

_GRUPO 7_

_Beatriz Herguedas Pinedo_

_Pablo Hernández Aguado_

## Implementación del problema.

In [80]:
# importamos las cosas que vamos a usar de aima
from search import *

class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



### Ejemplo:

In [81]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado meta que esperamos
estado = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# le decimos a aima que resuelva nuestro problema con el metodo de busqueda en
# amplitud
breadth_first_tree_search(estado).solution()


[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

## Análisis de los métodos de búsqueda.

Se procede a hacer un análisis del coste, tanto en tiempo como en espacio del problema de los misioneros, utilizando el estado inicial del ejemplo anterior.

Implementamos la clase Problema_con_Analizados y la función 'resuelve_misioneros' para obtener la información necesaria que buscamos (de forma análoga a como hemos hecho en el 8-puzzle).



In [82]:
class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        ## No implementamos la función que comrueba si el problema es resoluble.
        ## Como para el ejemplo sabemos que sí, devolvemos siempre True.
        
        return True

In [83]:
def resuelve_misioneros(estado_inicial, estado_final, algoritmo, h=None, p=True):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p = Problema_con_Analizados(ProblemaMisioneros(estado_inicial, estado_final))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
            
        if p:
            print("Solución: {0}".format(sol))
            print("Algoritmo: {0}".format(algoritmo.__name__))
            
            if h: 
                print("Heurística: {0}".format(h.__name__))
            else:
                pass
            
            print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
        
    else: 
        print("Este problema no tiene solucion. ")

In [84]:
E0 = (3, 3, 0)
F0 = (0, 0, 1)

#### Búsqueda en anchura.

In [39]:
resuelve_misioneros(E0, F0, breadth_first_graph_search)

Solución: [('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2))]
Algoritmo: breadth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 15


In [55]:
%%timeit
resuelve_misioneros(E0, F0, breadth_first_graph_search, None, False)

267 µs ± 3.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [41]:
## Búsqueda en anchura.
## 11 acciones
## 15 nodos analizados
## 267 microsegundos de media
## ------------------------------------------

#### Búsqueda en profundidad.

In [46]:
resuelve_misioneros(E0, F0, depth_first_graph_search)

Solución: [('1m1c', (1, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1m', (1, 0)), ('1m1c', (1, 1))]
Algoritmo: depth_first_graph_search
Longitud de la solución: 11. Nodos analizados: 12


In [59]:
%%timeit
resuelve_misioneros(E0, F0, depth_first_graph_search, None, False)

256 µs ± 3.68 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [51]:
## Búsqueda en profundidad.
## 11 acciones
## 12 nodos analizados
## 256 microsegundos de media
## ------------------------------------------

#### Búsqueda de coste uniforme.

In [52]:
resuelve_misioneros(E0, F0, uniform_cost_search)

Solución: [('1m1c', (1, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2))]
Algoritmo: uniform_cost_search
Longitud de la solución: 11. Nodos analizados: 15


In [64]:
%%timeit
resuelve_misioneros(E0, F0, uniform_cost_search, None, False)

402 µs ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
## Coste uniforme.
## 11 acciones
## 15 nodos analizados
## 402 microsegundos de media
## ------------------------------------------

#### Tabla de resultados.

|                    Método | Longitud | Nodos analizados | Tiempo (µs) |
|:--------------------------|:---------|:-----------------|:------------|
|                   Anchura |       11 |               15 |         267 |
|               Profundidad |       11 |               12 |         256 |
|            Coste Uniforme |       11 |               15 |         402 |

En efecto, todos los métodos nos devuelven la solución óptima al problema. Además, tanto la búsqueda en anchura como la búsqueda en profundidad marcan tiempos muy cercanos, casi la mitad de la marca de la búsqueda con coste uniforme. No obstante, el método de búsqueda en profundidad consigue alcanzar el resultado analizando 3 nodos menos (12) que el método de búsqueda por anchura (15).

### Ejercicio opcional. Define alguna heurística y estudia las propiedades del algoritmo A*

Definimos las heurísticas asociadas al problema descritas en los apuntes de la asignatura.

#### Heurística 1

Esta heurística resulta de suponer que siempre viajan un caníbal y un misionero juntos, con la acción 'Mover1M1C'.

In [85]:
def h1(node):
    state = node.state
    goal = (0, 0, 1)
    
    return (state[0] + state[1]) / 2

In [86]:
resuelve_misioneros(E0, F0, astar_search, h1)

Solución: [('1m1c', (1, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2))]
Algoritmo: astar_search
Heurística: h1
Longitud de la solución: 11. Nodos analizados: 14


In [87]:
%%timeit
resuelve_misioneros(E0, F0, astar_search, h1, False)

427 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Heurística 2

En esta heurística suponemos que los caníbales nunca se comen a los misioneros, luego la solución es que, en cada viaje de ida y vuelta, podemos transportar a una persona (pues la otra debe llevar de vuelta la barca).

Por tanto, la heurística es h(n) = 2*(c + m) - orilla(n), donde orilla(n) es 1 si la barca está en la orilla inicial y 0 en caso contrario.

In [77]:
def h2(node):
    state = node.state
    goal = (0, 0, 1)
    
    return 2 * (state[0] + state[1]) + state[2] - 1

In [78]:
resuelve_misioneros(E0, F0, astar_search, h2)

Solución: [('1m1c', (1, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('1c', (0, 1)), ('2m', (2, 0)), ('1m1c', (1, 1)), ('2m', (2, 0)), ('1c', (0, 1)), ('2c', (0, 2)), ('1c', (0, 1)), ('2c', (0, 2))]
Algoritmo: astar_search
Heurística: h2
Longitud de la solución: 11. Nodos analizados: 14


In [79]:
%%timeit
resuelve_misioneros(E0, F0, astar_search, h2, False)

421 µs ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Tabla actualizada.

|                    Método | Longitud | Nodos analizados | Tiempo (µs) |
|:--------------------------|:---------|:-----------------|:------------|
|                   Anchura |       11 |               15 |         267 |
|               Profundidad |       11 |               12 |         256 |
|            Coste Uniforme |       11 |               15 |         402 |
|              Heurística 1 |       11 |               14 |         427 |
|              Heurística 2 |       11 |               14 |         421 |

Las heurísticas no mejoran las búsquedas en profundidad y anchura. Esto es porque el espacio de estados es tan pequeño que estas funciones son más eficientes que la búsqueda A* con heurística.